In [2]:
%cd drive/My\ Drive/colab/dncnn

/content/drive/My Drive/colab/dncnn


In [3]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import time
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/vgg')

In [5]:
tflite_model = converter.convert()
tflite_models_dir = Path("tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"vgg.tflite"
tflite_model_file.write_bytes(tflite_model)

2669928

In [6]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"vgg_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

679968

In [7]:
!ls -lh {tflite_models_dir}

total 3.2M
-rw------- 1 root root 665K Jul 13 20:26 vgg_quant.tflite
-rw------- 1 root root 2.6M Jul 13 20:26 vgg.tflite


In [8]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [9]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [10]:
def gaussian_noise_layer(dim):
    std = 25#random.randint(0, 55)
    noise = tf.random.normal(shape=[dim, dim, 1], mean=0.0, stddev=std / 255.0, dtype=tf.float32)
    return noise

def augment_test(image):
    image = tf.io.read_file(image)
    image = img = tf.image.decode_png(image, channels=1)
    image = tf.image.resize_with_crop_or_pad(image, 180, 180)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    return image

def configure_for_test(ds):
    ds = ds.cache()
    ds = ds.map(augment_test, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(1)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [12]:
tf.random.set_seed(123)
test_dir = 'data/12/*.png'
test_ds = tf.data.Dataset.list_files(test_dir)
test_ds = configure_for_test(test_ds)
noises = [gaussian_noise_layer(180) for i in range(12)]


In [16]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interp, noises):
  input_index = interp.get_input_details()[0]["index"]
  output_index = interp.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  psnr = 0.0
  for i, image in enumerate(test_ds):
    noisy_image = image+noises[i]
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    #test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interp.set_tensor(input_index, noisy_image)

    # Run inference.
    interp.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interp.get_tensor(output_index)
    psnr += tf.image.psnr(noisy_image-output, image,  max_val=1.0)

  return psnr/(i+1)

In [21]:
psnr=evaluate_model(interpreter_quant, noises)
print(psnr)

21.08946204185486
tf.Tensor([27.551874], shape=(1,), dtype=float32)


In [22]:
psnr=evaluate_model(interpreter, noises)
print(psnr)

9.15105128288269
tf.Tensor([27.76517], shape=(1,), dtype=float32)
